<a href="https://colab.research.google.com/github/henry8695/Henry-Woodward/blob/master/Coronavirus_16_03_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up Environment

In [0]:
#Installing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

# Importing Files

In [2]:
#Importing Confirmed cases file
from google.colab import files
upload = files.upload()

Saving confirmed.csv to confirmed.csv


In [3]:
#Importing Deaths file
from google.colab import files
upload = files.upload()

Saving deaths.csv to deaths.csv


In [4]:
#Importing Population file
from google.colab import files
upload = files.upload()

Saving API_SP.POP.TOTL_DS2_en_csv_v2_821007.csv to API_SP.POP.TOTL_DS2_en_csv_v2_821007.csv


In [5]:
#Importing Population Density file
from google.colab import files
upload = files.upload()

Saving population density.csv to population density.csv


In [6]:
#Importing Urban Population file
from google.colab import files
upload = files.upload()

Saving API_SP.URB.TOTL.IN.ZS_DS2_en_csv_v2_821361.csv to API_SP.URB.TOTL.IN.ZS_DS2_en_csv_v2_821361.csv


# Setting up Dataframes

In [0]:
#Setting up dataframe
covid_cases = pd.read_csv("confirmed.csv")
covid_deaths = pd.read_csv("deaths.csv")
population = pd.read_csv("API_SP.POP.TOTL_DS2_en_csv_v2_821007.csv")
density = pd.read_csv("population density.csv")
urban = pd.read_csv("API_SP.URB.TOTL.IN.ZS_DS2_en_csv_v2_821361.csv")

# Pre-processing Covid-19 Cases for Original Model

In [0]:
covid_cases.head()

In [0]:
#Subsetting for minimum cases
covid_cases=covid_cases[(covid_cases["03/04/2020"]>30)]
covid_cases

In [0]:
#Renaming variables
covid_cases.rename(columns={"Country/Region":"Country"}, inplace=True)
covid_cases

In [0]:
#Taking out China and US
covid_cases = covid_cases[covid_cases.Country != "Mainland China"]
covid_cases = covid_cases[covid_cases.Country != "US"]
covid_cases = covid_cases[covid_cases.Country != "Others"]
covid_cases = covid_cases[covid_cases.Country != "Taipei and environs"]
covid_cases = covid_cases[covid_cases.Country != "China"]
covid_cases = covid_cases[covid_cases.Country != "Cruise Ship"]
covid_cases = covid_cases[covid_cases.Country != "Taiwan*"]
covid_cases

In [0]:
covid_cases.columns

In [0]:
#Taking out unnecessary variables
covid_cases.columns
covid_cases=covid_cases.drop(['Province/State','Lat', 'Long', '1/22/20', '1/23/20','1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20','1/30/20', '1/31/20', '02/01/2020', '02/02/2020', '02/03/2020','02/04/2020', '02/05/2020', '02/06/2020', '02/07/2020', '02/08/2020','02/09/2020', '02/10/2020', '02/11/2020', '02/12/2020', '2/13/20','2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20','2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20','2/26/20', '2/27/20', '2/28/20', '2/29/20', '03/01/2020', '03/02/2020','03/03/2020'], axis=1)
covid_cases

In [0]:
#Creating growth rate variable
covid_cases['Growth Rate']=covid_cases['03/11/2020']/covid_cases['03/04/2020']
covid_cases

In [0]:

covid_cases["Country"].unique()

# Pre-processing Population Density Data

In [18]:
#Inspecting population density dataframe 
density

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,ABW,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,307.988889,312.361111,314.972222,316.844444,318.666667,320.638889,322.527778,324.366667,326.255556,328.127778,330.222222,332.444444,334.683333,336.266667,336.983333,336.588889,335.366667,333.905556,333.222222,333.866667,336.483333,340.805556,345.561111,349.088889,350.144444,348.022222,343.516667,339.327778,339.066667,345.272222,359.011111,379.083333,402.800000,426.111111,446.244444,462.222222,474.727778,484.872222,494.472222,504.738889,516.100000,527.733333,538.983333,548.538889,555.727778,560.188889,562.344444,563.100000,563.638889,564.827778,566.922222,569.777778,573.105556,576.522222,579.672222,582.622222,585.366667,588.027778
1,Afghanistan,AFG,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,14.044987,14.323808,14.617537,14.926295,15.250314,15.585020,15.929795,16.293023,16.686236,17.114913,17.577191,18.060863,18.547565,19.013188,19.436265,19.825220,20.174779,20.435006,20.542009,20.458461,20.175341,19.732451,19.204316,18.693582,18.286015,17.976563,17.774920,17.795553,18.179820,19.012205,20.370396,22.187829,24.226638,26.155266,27.740491,28.878224,29.649735,30.232768,30.896125,31.829110,33.095898,34.618096,36.272510,37.874405,39.295220,40.488082,41.510486,42.462819,43.492959,44.704082,46.131503,47.730564,49.428038,51.114778,52.712071,54.197114,55.595993,56.937760
2,Angola,AGO,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,4.436891,4.498708,4.555593,4.600180,4.628676,4.637213,4.631622,4.629544,4.654892,4.724765,4.845413,5.012073,5.211328,5.423422,5.634074,5.839022,6.042941,6.249063,6.463517,6.690695,6.930654,7.181319,7.442124,7.712163,7.990693,8.277943,8.574035,8.877878,9.188078,9.503799,9.825059,10.152696,10.487727,10.831593,11.185695,11.551070,11.928748,12.320206,12.727095,13.151097,13.592487,14.052633,14.535557,15.046238,15.588034,16.162590,16.768559,17.402450,18.059101,18.734456,19.427818,20.139513,20.867715,21.610475,22.366553,23.135064,23.916538,24.713052
3,Albania,ALB,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,60.576642,62.456898,64.329234,66.209307,68.058066,69.874927,71.737153,73.805547,75.974270,77.937190,79.848650,81.865912,83.823066,85.770949,87.767555,89.727226,91.735255,93.659343,95.541314,97.518139,99.491095,101.615985,103.794161,106.001058,108.202993,110.315146,112.540328,114.683796,117.808139,119.946788,119.225912,118.505073,117.784197,117.063358,116.342482,115.621642,114.900766,114.179927,113.459051,112.738212,111.685146,111.350730,110.934891,110.472226,109.908285,109.217044,108.394781,107.566204,106.843759,106.314635,106.029015,105.854051,105.660292,105.441752,105.135146,104.967190,104.870693,104.612263
4,Andorra,AND,Population density (people per sq. km of land ...,EN.POP.DNST,NaN,30.585106,32.702128,34.919149,37.168085,39.465957,41.802128,44.165957,46.574468,49.059574,51.651064,54.380851,57.217021,60.068085,62.808511,65.329787,67.610638,69.725532,71.780851,74.080851,76.738298,79.787234,83.221277,86.951064,90.863830,94.893617,98.972340,103.095745,107.306383,111.591489,115.976596,120.576596,125.293617,129.725532,133.355319,135.851064,136.936170,136.865957,136.472340,136.957447,139.127660,143.278723,149.040425,155.706383,162.221277,167.802128,172.325532,175.923404,178.429787,179.708511,179.678723,178.185106,175.376596,171.859575,168.538298,165.980851,164.461702,163.831915,163.842553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [19]:
#Subsetting relevant variables
density=density.loc[:,["Country Name","2018"]]
density

,Country Name,2018
0,Aruba,588.027778
1,Afghanistan,56.937760
2,Angola,24.713052
3,Albania,104.612263
4,Andorra,163.842553
...,...,...
259,Kosovo,NaN
260,"Yemen, Rep.",53.977853
261,South Africa,47.630120
262,Zambia,23.341479


In [20]:
#Changing variable names
density.columns=["Country","Population Density"]
density

,Country,Population Density
0,Aruba,588.027778
1,Afghanistan,56.937760
2,Angola,24.713052
3,Albania,104.612263
4,Andorra,163.842553
...,...,...
259,Kosovo,NaN
260,"Yemen, Rep.",53.977853
261,South Africa,47.630120
262,Zambia,23.341479


In [21]:
#Inspecting country names to make sure they match covid_cases
density["Country"].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Albania', 'Andorra',
       'Arab World', 'United Arab Emirates', 'Argentina', 'Armenia',
       'American Samoa', 'Antigua and Barbuda', 'Australia', 'Austria',
       'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso',
       'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas, The',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei Darussalam', 'Bhutan',
       'Botswana', 'Central African Republic', 'Canada',
       'Central Europe and the Baltics', 'Switzerland', 'Channel Islands',
       'Chile', 'Mainland China', "Cote d'Ivoire", 'Cameroon',
       'Congo, Dem. Rep.', 'Congo, Rep.', 'Colombia', 'Comoros',
       'Cabo Verde', 'Costa Rica', 'Caribbean small states', 'Cuba',
       'Curacao', 'Cayman Islands', 'Cyprus', 'Czech Republic', 'Germany',
       'Djibouti', 'Dominica', 'Denmark', 'Dominican Republic', 'Algeria',
       'East Asia & Pacific (excluding high income)',

In [0]:
#Countries in covid_cases: 'Thailand', 'Japan', 'Singapore', 'Malaysia', 'Germany', 'Italy','Sweden', 'Spain', 'Iraq', 'Bahrain', 'Kuwait', 'Switzerland','Norway', 'Netherlands', 'Iran', 'Korea, South', 'France','Taiwan*', 'United Kingdom'
#NEED HELP RATHER THAN MANUALLY CHECKING
#Names needing to change to from density: 'Iran, Islamic Rep.' to 'Iran'; 'Korea, Rep.' to 'Korea, South'; 'UK' to 'United Kingdom'

In [23]:
density.loc[density["Country"] == "UK", "Country"] = "United Kingdom"
density.loc[density["Country"] == "Iran, Islamic Rep.", "Country"] = "Iran"
density.loc[density["Country"] == "Korea, Rep.", "Country"] = "Korea, South"
density["Country"].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Albania', 'Andorra',
       'Arab World', 'United Arab Emirates', 'Argentina', 'Armenia',
       'American Samoa', 'Antigua and Barbuda', 'Australia', 'Austria',
       'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso',
       'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas, The',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei Darussalam', 'Bhutan',
       'Botswana', 'Central African Republic', 'Canada',
       'Central Europe and the Baltics', 'Switzerland', 'Channel Islands',
       'Chile', 'Mainland China', "Cote d'Ivoire", 'Cameroon',
       'Congo, Dem. Rep.', 'Congo, Rep.', 'Colombia', 'Comoros',
       'Cabo Verde', 'Costa Rica', 'Caribbean small states', 'Cuba',
       'Curacao', 'Cayman Islands', 'Cyprus', 'Czech Republic', 'Germany',
       'Djibouti', 'Dominica', 'Denmark', 'Dominican Republic', 'Algeria',
       'East Asia & Pacific (excluding high income)',

In [24]:
#Removing Outliers
density=density[density["Population Density"] < 1000]
density

,Country,Population Density
0,Aruba,588.027778
1,Afghanistan,56.937760
2,Angola,24.713052
3,Albania,104.612263
4,Andorra,163.842553
...,...,...
258,Samoa,69.303887
260,"Yemen, Rep.",53.977853
261,South Africa,47.630120
262,Zambia,23.341479


In [25]:
#Merging with covid_cases
left=covid_cases
right=density
covid_cases=pd.merge(left, right, on="Country")
covid_cases

,Country,03/04/2020,03/05/2020,03/06/2020,03/07/2020,03/08/2020,03/09/2020,03/10/2020,03/11/2020,03/12/2020,3/13/20,3/14/20,Growth Rate,Population Density
0,Thailand,43,47,48,50,50,50,53,59,70,75,82,1.372093,135.897207
1,Japan,331,360,420,461,502,511,581,639,639,701,773,1.930514,347.073458
2,Malaysia,50,50,83,93,99,117,129,149,149,197,238,2.980000,95.962821
3,Germany,262,482,670,799,1040,1176,1457,1908,2078,3675,4585,7.282443,237.370970
4,Italy,3089,3858,4636,5883,7375,9172,10149,12462,12462,17660,21157,4.034315,205.450748
5,Sweden,35,94,101,161,203,248,355,500,599,814,961,14.285714,25.001043
6,Spain,222,259,400,500,673,1073,1695,2277,2277,5232,6391,10.256757,93.529058
7,Iraq,35,35,40,54,60,60,71,71,71,101,110,2.028571,88.530570
8,Kuwait,56,58,58,61,64,64,69,72,80,80,104,1.285714,232.172222
9,Switzerland,90,114,214,268,337,374,491,652,652,1139,1359,7.244444,215.521378


# Pre-processing Population Data

In [26]:
#Inspecting population dataframe 
population

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,57715.0,58055.0,58386.0,58726.0,59063.0,59440.0,59840.0,60243.0,60528.0,60657.0,60586.0,60366.0,60103.0,59980.0,60096.0,60567.0,61345.0,62201.0,62836.0,63026.0,62644.0,61833.0,61079.0,61032.0,62149.0,64622.0,68235.0,72504.0,76700.0,80324.0,83200.0,85451.0,87277.0,89005.0,90853.0,92898.0,94992.0,97017.0,98737.0,100031.0,100834.0,101222.0,101358.0,101455.0,101669.0,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,10174836.0,10399926.0,10637063.0,10893776.0,11173642.0,11475445.0,11791215.0,12108963.0,12412950.0,12689160.0,12943093.0,13171306.0,13341198.0,13411056.0,13356511.0,13171673.0,12882528.0,12537730.0,12204292.0,11938208.0,11736179.0,11604534.0,11618005.0,11868877.0,12412308.0,13299017.0,14485546.0,15816603.0,17075727.0,18110657.0,18853437.0,19357126.0,19737765.0,20170844.0,20779953.0,21606988.0,22600770.0,23680871.0,24726684.0,25654277.0,26433049.0,27100536.0,27722276.0,28394813.0,29185507.0,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,5781214.0,5774243.0,5771652.0,5803254.0,5890365.0,6040777.0,6248552.0,6496962.0,6761380.0,7024000.0,7279509.0,7533735.0,7790707.0,8058067.0,8341289.0,8640446.0,8952950.0,9278096.0,9614754.0,9961997.0,10320111.0,10689250.0,11068050.0,11454777.0,11848386.0,12248901.0,12657366.0,13075049.0,13503747.0,13945206.0,14400719.0,14871570.0,15359601.0,15866869.0,16395473.0,16945753.0,17519417.0,18121479.0,18758145.0,19433602.0,20149901.0,20905363.0,21695634.0,22514281.0,23356246.0,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,2022272.0,2081695.0,2135479.0,2187853.0,2243126.0,2296752.0,2350124.0,2404831.0,2458526.0,2513546.0,2566266.0,2617832.0,2671997.0,2726056.0,2784278.0,2843960.0,2904429.0,2964762.0,3022635.0,3083605.0,3142336.0,3227943.0,3286542.0,3266790.0,3247039.0,3227287.0,3207536.0,3187784.0,3168033.0,3148281.0,3128530.0,3108778.0,3089027.0,3060173.0,3051010.0,3039616.0,3026939.0,3011487.0,2992547.0,2970017.0,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,19647.0,20758.0,21890.0,23058.0,24276.0,25559.0,26892.0,28232.0,29520.0,30705.0,31777.0,32771.0,33737.0,34818.0,36067.0,37500.0,39114.0,40867.0,42706.0,44600.0,46517.0,48455.0,50434.0,52448.0,54509.0,56671.0,58888.0,60971.0,62677.0,63850.0,64360.0,64327.0,64142.0,64370.0,65390.0,67341.0,70049.0,73182.0,76244.0,78867.0,80993.0,82684.0,83862.0,84463.0,84449.0,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,1106000.0,1135000.0,1163000.0,1191000.0,1219000.0,1247000.0,1278000.0,1308000.0,1339000.0,1369000.0,1400000.0,1430000.0,1460000.0,1491000.0,1521000.0,1552000.0,1582000.0,1614000.0,1647000.0,1682000.0,1717000.0,1753

In [0]:
#Subsetting relevant variables
population=population.loc[:,["Country Name","2018"]]

In [0]:
#Changing variable names
population.columns=["Country","Population"]

In [29]:
#Changing country names
population.loc[population["Country"] == "UK", "Country"] = "United Kingdom"
population.loc[population["Country"] == "Iran, Islamic Rep.", "Country"] = "Iran"
population.loc[population["Country"] == "Korea, Rep.", "Country"] = "Korea, South"
population["Country"].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Albania', 'Andorra',
       'Arab World', 'United Arab Emirates', 'Argentina', 'Armenia',
       'American Samoa', 'Antigua and Barbuda', 'Australia', 'Austria',
       'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso',
       'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas, The',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei Darussalam', 'Bhutan',
       'Botswana', 'Central African Republic', 'Canada',
       'Central Europe and the Baltics', 'Switzerland', 'Channel Islands',
       'Chile', 'China', "Cote d'Ivoire", 'Cameroon', 'Congo, Dem. Rep.',
       'Congo, Rep.', 'Colombia', 'Comoros', 'Cabo Verde', 'Costa Rica',
       'Caribbean small states', 'Cuba', 'Curacao', 'Cayman Islands',
       'Cyprus', 'Czech Republic', 'Germany', 'Djibouti', 'Dominica',
       'Denmark', 'Dominican Republic', 'Algeria',
       'East Asia & Pacific (excluding high income)',
       '

In [30]:
#Merging with covid_cases
left=covid_cases
right=population
covid_cases=pd.merge(left, right, on="Country")
covid_cases

,Country,03/04/2020,03/05/2020,03/06/2020,03/07/2020,03/08/2020,03/09/2020,03/10/2020,03/11/2020,03/12/2020,3/13/20,3/14/20,Growth Rate,Population Density,Population
0,Thailand,43,47,48,50,50,50,53,59,70,75,82,1.372093,135.897207,69428524.0
1,Japan,331,360,420,461,502,511,581,639,639,701,773,1.930514,347.073458,126529100.0
2,Malaysia,50,50,83,93,99,117,129,149,149,197,238,2.980000,95.962821,31528585.0
3,Germany,262,482,670,799,1040,1176,1457,1908,2078,3675,4585,7.282443,237.370970,82927922.0
4,Italy,3089,3858,4636,5883,7375,9172,10149,12462,12462,17660,21157,4.034315,205.450748,60431283.0
5,Sweden,35,94,101,161,203,248,355,500,599,814,961,14.285714,25.001043,10183175.0
6,Spain,222,259,400,500,673,1073,1695,2277,2277,5232,6391,10.256757,93.529058,46723749.0
7,Iraq,35,35,40,54,60,60,71,71,71,101,110,2.028571,88.530570,38433600.0
8,Kuwait,56,58,58,61,64,64,69,72,80,80,104,1.285714,232.172222,4137309.0
9,Switzerland,90,114,214,268,337,374,491,652,652,1139,1359,7.244444,215.521378,8516543.0


# Pre-processing Urban Population Data

In [31]:
#Inspecting population dataframe 
urban

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,Urban population (% of total population),SP.URB.TOTL.IN.ZS,50.776,50.761,50.746,50.730,50.715,50.700,50.685,50.670,50.654,50.639,50.624,50.609,50.593,50.578,50.563,50.548,50.532,50.517,50.502,50.487,50.472,50.456,50.441,50.426,50.411,50.395,50.380,50.365,50.350,50.335,50.319,50.304,49.998,49.588,49.177,48.767,48.356,47.946,47.536,47.127,46.717,46.339,45.972,45.606,45.240,44.875,44.511,44.147,43.783,43.421,43.059,42.940,42.957,42.990,43.041,43.108,43.192,43.293,43.411,NaN
1,Afghanistan,AFG,Urban population (% of total population),SP.URB.TOTL.IN.ZS,8.401,8.684,8.976,9.276,9.586,9.904,10.233,10.570,10.918,11.275,11.643,12.021,12.410,12.809,13.219,13.641,14.074,14.517,14.973,15.442,15.995,16.562,17.147,17.747,18.365,18.997,19.647,20.314,20.998,21.088,21.177,21.266,21.355,21.444,21.534,21.624,21.714,21.805,21.895,21.986,22.078,22.169,22.261,22.353,22.500,22.703,22.907,23.113,23.320,23.528,23.737,23.948,24.160,24.373,24.587,24.803,25.020,25.250,25.495,NaN
2,Angola,AGO,Urban population (% of total population),SP.URB.TOTL.IN.ZS,10.435,10.798,11.204,11.624,12.058,12.504,12.965,13.441,13.932,14.436,14.957,15.632,16.455,17.310,18.202,19.128,20.092,21.088,22.122,23.192,24.298,25.437,26.612,27.821,29.065,30.338,31.643,32.978,34.343,35.731,37.144,38.580,40.039,41.511,43.000,44.169,45.346,46.525,47.710,48.897,50.087,51.274,52.461,53.645,54.827,56.000,56.764,57.524,58.282,59.034,59.783,60.528,61.268,62.002,62.731,63.446,64.149,64.839,65.514,NaN
3,Albania,ALB,Urban population (% of total population),SP.URB.TOTL.IN.ZS,30.705,30.943,31.015,31.086,31.158,31.230,31.302,31.374,31.446,31.548,31.740,31.933,32.127,32.321,32.516,32.712,32.908,33.105,33.302,33.500,33.762,34.024,34.287,34.552,34.817,35.083,35.350,35.619,35.888,36.158,36.428,36.700,37.249,37.799,38.354,38.911,39.473,40.035,40.601,41.169,41.741,42.435,43.501,44.573,45.651,46.731,47.815,48.902,49.991,51.076,52.163,53.247,54.330,55.387,56.423,57.434,58.421,59.383,60.319,NaN
4,Andorra,AND,Urban population (% of total population),SP.URB.TOTL.IN.ZS,58.450,60.983,63.462,65.872,68.205,70.445,72.593,74.641,76.588,78.424,80.155,81.780,83.301,84.716,86.032,87.252,88.382,89.420,90.378,91.257,92.064,92.800,93.474,94.089,94.649,95.159,95.325,95.178,95.027,94.872,94.712,94.530,94.324,94.111,93.890,93.661,93.425,93.180,92.927,92.666,92.395,92.056,91.641,91.207,90.751,90.285,89.807,89.308,89.004,88.912,88.819,88.726,88.631,88.537,88.441,88.345,88.248,88.150,88.062,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,Urban population (% of total population),SP.URB.TOTL.IN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,"Yemen, Rep.",YEM,Urban population (% of total population),SP.URB.TOTL.IN.ZS,9.100,9.459,9.831,10.216,10.614,11.026,11.451,11.891,12.347,12.815,13.300,13.598,13.901,14.209,14.524,14.844,15.171,15.502,15.840,16.183,16.534,16.889,17.251,17.619,17.993,18.373,18.808,19.323,19.849,20.385,20.931,21.489,22.058,22.636,23.225,23.760,24.249,24.743,25.244,25.752,26.267,26.787,27.315,27.849,28.390,28.936,29.490,30.051,30.619,31.194,31.776,32.364,32.958,33.559,34.165,34.777,35.394,36.016,36.642,NaN
261,South Africa,ZAF,Urban population (% of total population),SP.URB.TOTL.IN.ZS,46.619,46.793,46.906,47.020,47.134,47.248,47.362,

In [0]:
#Subsetting relevant variables
urban=urban.loc[:,["Country Name","2018"]]

In [0]:
#Changing variable names
urban.columns=["Country","Urban"]

In [34]:
#Changing country names
urban.loc[urban["Country"] == "UK", "Country"] = "United Kingdom"
urban.loc[urban["Country"] == "Iran, Islamic Rep.", "Country"] = "Iran"
urban.loc[urban["Country"] == "Korea, Rep.", "Country"] = "Korea, South"
urban["Country"].unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Albania', 'Andorra',
       'Arab World', 'United Arab Emirates', 'Argentina', 'Armenia',
       'American Samoa', 'Antigua and Barbuda', 'Australia', 'Austria',
       'Azerbaijan', 'Burundi', 'Belgium', 'Benin', 'Burkina Faso',
       'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas, The',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bermuda',
       'Bolivia', 'Brazil', 'Barbados', 'Brunei Darussalam', 'Bhutan',
       'Botswana', 'Central African Republic', 'Canada',
       'Central Europe and the Baltics', 'Switzerland', 'Channel Islands',
       'Chile', 'China', "Cote d'Ivoire", 'Cameroon', 'Congo, Dem. Rep.',
       'Congo, Rep.', 'Colombia', 'Comoros', 'Cabo Verde', 'Costa Rica',
       'Caribbean small states', 'Cuba', 'Curacao', 'Cayman Islands',
       'Cyprus', 'Czech Republic', 'Germany', 'Djibouti', 'Dominica',
       'Denmark', 'Dominican Republic', 'Algeria',
       'East Asia & Pacific (excluding high income)',
       '

In [35]:
#Merging with covid_cases
left=covid_cases
right=urban
covid_cases=pd.merge(left, right, on="Country")
covid_cases

,Country,03/04/2020,03/05/2020,03/06/2020,03/07/2020,03/08/2020,03/09/2020,03/10/2020,03/11/2020,03/12/2020,3/13/20,3/14/20,Growth Rate,Population Density,Population,Urban
0,Thailand,43,47,48,50,50,50,53,59,70,75,82,1.372093,135.897207,69428524.0,49.949
1,Japan,331,360,420,461,502,511,581,639,639,701,773,1.930514,347.073458,126529100.0,91.616
2,Malaysia,50,50,83,93,99,117,129,149,149,197,238,2.980000,95.962821,31528585.0,76.036
3,Germany,262,482,670,799,1040,1176,1457,1908,2078,3675,4585,7.282443,237.370970,82927922.0,77.312
4,Italy,3089,3858,4636,5883,7375,9172,10149,12462,12462,17660,21157,4.034315,205.450748,60431283.0,70.438
5,Sweden,35,94,101,161,203,248,355,500,599,814,961,14.285714,25.001043,10183175.0,87.431
6,Spain,222,259,400,500,673,1073,1695,2277,2277,5232,6391,10.256757,93.529058,46723749.0,80.321
7,Iraq,35,35,40,54,60,60,71,71,71,101,110,2.028571,88.530570,38433600.0,70.473
8,Kuwait,56,58,58,61,64,64,69,72,80,80,104,1.285714,232.172222,4137309.0,100.000
9,Switzerland,90,114,214,268,337,374,491,652,652,1139,1359,7.244444,215.521378,8516543.0,73.797


# Creating Temperature Dataframe

In [0]:
covid_cases

In [37]:
covid_cases["Country"].unique()

array(['Thailand', 'Japan', 'Malaysia', 'Germany', 'Italy', 'Sweden',
       'Spain', 'Iraq', 'Kuwait', 'Switzerland', 'Norway', 'Netherlands',
       'Iran', 'Korea, South', 'France', 'United Kingdom'], dtype=object)

In [0]:
d = {'Country': ['Thailand', 'Japan', 'Malaysia', 'Germany', 'Italy', 'Sweden','Spain', 'Iraq', 'Kuwait', 'Switzerland', 'Norway', 'Netherlands','Iran', 'Korea, South', 'France', 'United Kingdom'], 'Temperature': [301, 282, 300, 275, 282, 270, 280, 286, 288, 275, 274, 276, 275, 272, 277, 278]}
temperature = pd.DataFrame(data=d)
temperature

In [0]:
#Merging with covid_cases
left=covid_cases
right=temperature
covid_cases=pd.merge(left, right, on="Country")
covid_cases

# Country Comparison Graph

In [0]:
graph = covid_cases.T
graph.columns=graph.iloc[0]
graph=graph[1:-5]
graph.plot()

# Government Performance

In [0]:
#Pre-processing deaths dataset
covid_deaths.rename(columns={"Country/Region":"Country"}, inplace=True)
left=covid_cases["Country"]
right=covid_deaths
cases_deaths=pd.merge(left, right, on="Country")
cases_deaths=cases_deaths[(cases_deaths["3/14/20"]>20)]
cases_deaths=cases_deaths.drop(['Province/State','Lat', 'Long', '1/22/20', '1/23/20','1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20','1/30/20', '1/31/20', '02/01/2020', '02/02/2020', '02/03/2020','02/04/2020', '02/05/2020', '02/06/2020', '02/07/2020', '02/08/2020','02/09/2020', '02/10/2020', '02/11/2020', '02/12/2020', '2/13/20','2/14/20', '2/15/20', '2/16/20', '2/17/20', '2/18/20', '2/19/20','2/20/20', '2/21/20', '2/22/20', '2/23/20', '2/24/20', '2/25/20','2/26/20', '2/27/20', '2/28/20', '2/29/20', '03/01/2020', '03/02/2020','03/03/2020'], axis=1)
cases_deaths

In [0]:
#Calculating Death Rate
covid_cases["Death Rate"]=cases_deaths["3/14/20"]/covid_cases["3/14/20"]
covid_cases["Death Rate"]

In [0]:
#Predicting case numer
covid_cases["Predicted Cases"]=covid_cases["03/11/2020"]*covid_cases["Growth Rate"]**(3/7)

In [0]:
Gov_impact = (covid_cases["Country"], covid_cases["3/14/20"], covid_cases["Predicted Cases"], covid_cases["Death Rate"])
Gov_impact = pd.DataFrame(data=Gov_impact)
Gov_impact = Gov_impact.T
Gov_impact

In [0]:
#Calculating proxy for government impact
Gov_impact["Government Impact"]=Gov_impact["3/14/20"]-Gov_impact["Predicted Cases"]
Gov_impact["Government Impact"]

In [0]:
#Visualising Government Impact on Confirmed Cases
x=Gov_impact["Country"]
y=Gov_impact["Government Impact"]/Gov_impact["3/14/20"]
plt.bar(x,y)
plt.xlabel('Country', fontsize=10)
plt.ylabel('Government Impact', fontsize=10)
plt.xticks(x, fontsize=10, rotation=45)
plt.title('Government Impact on Confirmed Cases')

In [0]:
#Visualising Government Impact on Lives
x=Gov_impact["Country"]
y=Gov_impact["Government Impact"]*Gov_impact["Death Rate"]/Gov_impact['3/14/20']*1000
plt.bar(x,y)
plt.xlabel('Country', fontsize=10)
plt.ylabel('Government Impact', fontsize=10)
plt.xticks(x, fontsize=10, rotation=45)
plt.title('Government Impact on Lives Saved')

# Creating Growth Rate Dataframe 


In [0]:
covid_cases

In [0]:
Country = covid_cases["Country"]
Growth_Rate = pd.DataFrame(data=Country)
Growth_Rate

In [0]:
Growth_Rate["Growth_Rate1"] = covid_cases['03/11/2020']/covid_cases['03/04/2020']
Growth_Rate["Growth_Rate2"] = covid_cases['03/12/2020']/covid_cases['03/05/2020']
Growth_Rate["Growth_Rate3"] = covid_cases['3/13/20']/covid_cases['03/06/2020']
Growth_Rate["Growth_Rate4"] = covid_cases['3/14/20']/covid_cases['03/07/2020']

In [0]:
Growth_Rate

In [0]:
Graph = Growth_Rate.T
Graph.columns=Graph.iloc[0]
Graph=Graph[1:]
Graph.plot()

In [0]:
Graph = pd.DataFrame(data=Graph)

In [0]:
Graph=Graph.drop(["Thailand", "Japan", "Malaysia", "Iraq", "Kuwait", "Iran", "Korea, South"], axis=1)

In [0]:
Graph.plot()